<a href="https://colab.research.google.com/github/poddardelhi/crm_homeTask/blob/main/HomeTask_CRMAnalyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.set_project('crm-analyst-home-task')
jovian.set_colab_id('19qlyMoLkYQQTuyeGOT8H77D-46PRtm_K')

# Commit the work

In [79]:
#Save the work to JOVIAN.ML
project_name = "crm-analyst-home-task"
 # change this
jovian.commit(project=project_name)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ai/poddardelhi/crm-analyst-home-task


'https://jovian.ai/poddardelhi/crm-analyst-home-task'

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
# Importing the necessary libraries
import numpy as np
import pandas as pd
import pandas_profiling
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# Task 1



1. Calculate the following rates, based on data from the page "1. Data" (in the data sheet there are columns currently empty - fill them):
  
  1.1. Funnel Start Rate (Rate at which Sessions become Started Applications).
  
  1.2. Approval Rate (Rate at which Finished Applications become Approved Applications)
  
  1.3. Write Rate (Rate at which Approved Applications become Paid Applications)






In [47]:
df_application= pd.read_csv("/content/drive/MyDrive/Jovian/Ferratum_CRM/data_q1.csv")

In [48]:
df_application.head()

,Country,Date,Date_HR,Sessions,StartedApplications,FinishedApps,Approved,PaidOut
0,Australia,43466,01/01/2019,91127.0,28144.0,10587,523.0,510.0
1,Australia,43497,01/02/2019,77586.0,24290.0,9054,546.0,537.0
2,Australia,43525,01/03/2019,65420.0,18904.0,7614,446.0,439.0
3,Brazil,43466,01/01/2019,232295.0,143099.0,14264,1229.0,678.0
4,Brazil,43497,01/02/2019,168190.0,99253.0,14090,755.0,321.0


In [49]:
df_application.shape

(66, 8)

In [50]:
#checking the Null or Nan values
df_application.isna().sum()

Country                0
Date                   0
Date_HR                0
Sessions               3
StartedApplications    3
FinishedApps           0
Approved               2
PaidOut                2
dtype: int64

In [51]:
# Finding the individual rows that are Null/NaN type
none_values=df_application[df_application.isna().any(axis=1)][::]
none_values.head()


,Country,Date,Date_HR,Sessions,StartedApplications,FinishedApps,Approved,PaidOut
39,Nigeria,43466,01/01/2019,NaN,NaN,92940,6.0,4.0
40,Nigeria,43497,01/02/2019,NaN,NaN,70493,4.0,1.0
41,Nigeria,43525,01/03/2019,NaN,NaN,60915,5.0,2.0
52,Russia,43497,01/02/2019,38605.0,13951.0,6228,NaN,NaN
53,Russia,43525,01/03/2019,16772.0,389.0,0,NaN,NaN


## Empty columns?
We have many empty columns for countries Nigeria and Russia. I'm taking a assumption here that every time a user logs in, it is recorded as 1 session. Therefore, I'm disregarding Nigeria during analysis and value 0 is replaced for all empty cells.

In case of Russia, all NaN cells are replaced with value 0


In [52]:
df_application.fillna(value=0, axis=1, inplace=True)
df_application.isna().sum()

Country                0
Date                   0
Date_HR                0
Sessions               0
StartedApplications    0
FinishedApps           0
Approved               0
PaidOut                0
dtype: int64

In [53]:
df_application.head()

,Country,Date,Date_HR,Sessions,StartedApplications,FinishedApps,Approved,PaidOut
0,Australia,43466,01/01/2019,91127.0,28144.0,10587,523.0,510.0
1,Australia,43497,01/02/2019,77586.0,24290.0,9054,546.0,537.0
2,Australia,43525,01/03/2019,65420.0,18904.0,7614,446.0,439.0
3,Brazil,43466,01/01/2019,232295.0,143099.0,14264,1229.0,678.0
4,Brazil,43497,01/02/2019,168190.0,99253.0,14090,755.0,321.0


In [54]:
# dropping excel serial date and renaming the column Date_HR to Date
df_application.drop(labels='Date', axis=1,inplace=True)
df_application.rename(columns={'Date_HR':'Date'},inplace=True)
df_application.head()

,Country,Date,Sessions,StartedApplications,FinishedApps,Approved,PaidOut
0,Australia,01/01/2019,91127.0,28144.0,10587,523.0,510.0
1,Australia,01/02/2019,77586.0,24290.0,9054,546.0,537.0
2,Australia,01/03/2019,65420.0,18904.0,7614,446.0,439.0
3,Brazil,01/01/2019,232295.0,143099.0,14264,1229.0,678.0
4,Brazil,01/02/2019,168190.0,99253.0,14090,755.0,321.0


In [78]:
#Funnel Start Rate
df_application['FunnelStartRate']= (df_application['StartedApplications']/ df_application['Sessions']) * 100
#Conversion rate
df_application['ConversionRate']= (df_application['FinishedApps']/ df_application['StartedApplications']) * 100
#Approval rate
df_application['ApprovalRate']= (df_application['Approved']/df_application['FinishedApps']) *100
#Displaying the above calculated values
df_application.loc[:5,['Country','Date','FunnelStartRate','ConversionRate','ApprovalRate']]

,Country,Date,FunnelStartRate,ConversionRate,ApprovalRate
0,Australia,01/01/2019,30.884370,37.617254,4.940021
1,Australia,01/02/2019,31.307195,37.274599,6.030484
2,Australia,01/03/2019,28.896362,40.277190,5.857631
3,Brazil,01/01/2019,61.602273,9.967924,8.616096
4,Brazil,01/02/2019,59.012426,14.196044,5.358410
5,Brazil,01/03/2019,47.402617,24.424740,4.333376
